In [21]:
%%time
import numpy as np
import pandas as pd
import os
ss_token = os.getenv('smartsheet_token')
pd.options.mode.chained_assignment = None  # get rid of the "A value is trying to be set on a copy of a slice from a DataFrame." warning
## https://libraries.io/pypi/smartsheet-dataframe
from smartsheet_dataframe import get_as_df, get_report_as_df
## Data from this sheet ==> https://app.smartsheet.com/folders/5x6VP8Hqc8hMF6c5jr32frcmQHq7r9vGxpcRxWj1
df_original = get_report_as_df(token=ss_token,report_id=4674558964852612)
df = df_original[['Job#','Primary','Deadline','Word Count','Data Type','Assigned To']]
df.loc[(df['Data Type']=='To Client')|(df['Data Type']=='To Developer')|(df['Data Type']=='Delivery'),'Deliveries'] = 1
df.loc[(df['Data Type']!='To Client')&(df['Data Type']!='To Developer')&(df['Data Type']!='Delivery'),'Deliveries'] = 0
df.loc[(df['Data Type']=='PL'),'Placing (LPs)'] = 1
df.loc[(df['Data Type']!='PL'),'Placing (LPs)'] = 0
df.loc[(df['Data Type']=='FE')&(df['Primary']=='FE: FT FE'),'FT FE (words)']=df['Word Count']
df.loc[(df['Data Type']=='FE')&(df['Primary']=='FE: BT FE'),'BT FE (words)']=df['Word Count']
df.loc[(df['Data Type']=='FE')&(df['Primary']=='FE: SSR Round 1'),'R1 SSR FE (words)']=df['Word Count']
df.loc[(df['Data Type']=='FE')&(df['Primary']=='FE: SSR Round 2'),'R2 SSR FE (words)']=df['Word Count']
df.loc[(df['Data Type']=='FE')&(df['Primary']=='FE: SSR Round 3'),'R3 SSR FE (words)']=df['Word Count']
df.loc[(df['Data Type']=='PF'),'Preflight (jobs)']=df['Job#'].nunique()
df.loc[(df['Assigned To']=='cwan@transperfect.com'),'Assigned To']='Carmen Wan'
df.loc[(df['Assigned To']=='cma@transperfect.com'),'Assigned To']='Crystal Ma'
df.fillna(0)
capacity = df.groupby(['Deadline','Assigned To']).agg({'Deliveries':np.sum,'Preflight (jobs)':pd.Series.nunique,'Placing (LPs)':np.sum,'FT FE (words)':np.sum,'BT FE (words)':np.sum,'R1 SSR FE (words)':np.sum,'R2 SSR FE (words)':np.sum,'R3 SSR FE (words)':np.sum}).astype(int)

CPU times: total: 109 ms
Wall time: 1.16 s


In [22]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as ipw

def PM_Capacity (PM):
    capacity_PM = df.loc[df['Assigned To']==PM].groupby(['Deadline']).agg({'Deliveries':np.sum,'Preflight (jobs)':pd.Series.nunique,'Placing (LPs)':np.sum,'FT FE (words)':np.sum,'BT FE (words)':np.sum,'R1 SSR FE (words)':np.sum,'R2 SSR FE (words)':np.sum,'R3 SSR FE (words)':np.sum}).astype(int)
    return capacity_PM

interact(PM_Capacity,PM=["Carmen Wan","Crystal Ma"])

interactive(children=(Dropdown(description='PM', options=('Carmen Wan', 'Crystal Ma'), value='Carmen Wan'), Ou…

<function __main__.PM_Capacity(PM)>

In [23]:
!pip freeze > requirements.txt